In [6]:
%load_ext autoreload
%autoreload 2
from raw_data_processing import process_PAMAP2_all_data, process_hhar_all_files, process_motion_sense_all_files
import numpy as np
from sklearn.impute import KNNImputer
import pandas as pd
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
pamap_df = process_PAMAP2_all_data("test_run/original_datasets/PAMAP2")
hhar_df = process_hhar_all_files("test_run/original_datasets/hhar/Activity recognition exp")
motion_sense_df = process_motion_sense_all_files("test_run/original_datasets/motionsense/Data/")

108 (1218683, 3)
109 (25368, 3)
107 (937997, 3)
106 (1082715, 3)
104 (985200, 3)
105 (1120516, 3)
101 (1125961, 3)
102 (1335439, 3)
103 (757267, 3)
108 (1218683, 3)
109 (25368, 3)
107 (937997, 3)
106 (1082715, 3)
104 (985200, 3)
105 (1120516, 3)
101 (1125961, 3)
102 (1335439, 3)
103 (757267, 3)
108 (1218683, 3)
109 (25368, 3)
107 (937997, 3)
106 (1082715, 3)
104 (985200, 3)
105 (1120516, 3)
101 (1125961, 3)
102 (1335439, 3)
103 (757267, 3)
108 (3656049, 3)
109 (76104, 3)
107 (2813991, 3)
106 (3248145, 3)
104 (2955600, 3)
105 (3361548, 3)
101 (3377883, 3)
102 (4006317, 3)
103 (2271801, 3)
a (1665085, 3)
b (1743413, 3)
c (1636308, 3)
d (1624117, 3)
e (1812440, 3)
f (1657155, 3)
g (1396828, 3)
h (1349133, 3)
i (1415401, 3)
a (1732665, 3)
b (1843730, 3)
c (1701482, 3)
d (1713048, 3)
e (1878796, 3)
f (1562407, 3)
g (1453907, 3)
h (1423950, 3)
i (1488022, 3)
a (3397750, 3)
b (3587143, 3)
c (3337790, 3)
d (3337165, 3)
e (3691236, 3)
f (3219562, 3)
g (2850735, 3)
h (2773083, 3)
i (2903423, 3)


In [8]:
# pickle the datasets
with open('pickled_datasets/pamap2.pickle', 'wb') as file:
    pickle.dump(pamap_df, file)
with open('pickled_datasets/hhar2.pickle', 'wb') as file:
    pickle.dump(hhar_df, file)
with open('pickled_datasets/motionsense2.pickle', 'wb') as file:
    pickle.dump(motion_sense_df, file)

In [12]:
def concat_datasets(datasets, sensor_type):
    concated_datasets = {}
    ##TODO need to check key to see whether mag exists
    for df in datasets:
        concated_datasets.update(df[sensor_type])
    return concated_datasets

cdf = concat_datasets([pamap_df, hhar_df, motion_sense_df], "all")
def get_labels(data):
    all_labels = []
    for user in data:
        all_labels = np.concatenate((np.unique(data[user][0][1]), all_labels))
    labels = np.unique(all_labels)
    return labels

labels = get_labels(cdf)

In [13]:
print(labels)

['IMU ankle acc' 'IMU ankle gyro' 'IMU ankle mag' 'IMU chest acc'
 'IMU chest gyro' 'IMU chest mag' 'IMU hand acc' 'IMU hand gyro'
 'IMU hand mag' 'Phone acc' 'Phone gyro' 'Watch acc' 'Watch gyro'
 'iphone Acc']


In [ ]:
hhar_df_acc = hhar_df['acc']


In [ ]:
df = pd.DataFrame.from_records(hhar_df_acc)


In [21]:
import os
def process_HARTH_back_acc_data(data_folder_path):
    combined_data = []
    for filename in os.listdir(data_folder_path):
            file_path = os.path.join(data_folder_path, filename)
            df = pd.read_csv(file_path, sep=',', header=0, na_filter="NaN")
            df['User-ID'] = filename[1:4]
            combined_data.append(df)
    df = pd.concat(combined_data)
    df = df.replace(1, "walking")
    df = df.replace(2, "running")
    df = df.replace(3, "shuffling")
    df = df.replace(4, "stairs (ascending)")
    df = df.replace(5, "stairs (descending)")
    df = df.replace(6, "standing")
    df = df.replace(7, "sitting")
    df = df.replace(8, "lying")
    df = df.replace(13, "cycling (sit)")
    df = df.replace(14, "cycling (stand)")
    df = df.replace(130, "cycling (sit, inactive)")
    df = df.replace(140, "cycling (stand, inactive)")
    back_data = df.loc[:, ['User-ID', 'back_x', 'back_y', 'back_z', 'label']]
    harth_user = back_data["User-ID"].unique()


    user_datasets = {}
    for user in harth_user:
        user_extract = back_data[back_data['User-ID'] == user]
        data = user_extract[['back_x', 'back_y', 'back_z']].values
        labels = user_extract["label"].values
        print(f"{user} {data.shape}")
        user_datasets[user] = [(data, labels)]
    return user_datasets

combined_data = process_HARTH_back_acc_data("test_run/original_datasets/harth")

016 (355418, 3)
017 (366609, 3)
029 (178716, 3)
015 (418392, 3)
014 (366487, 3)
028 (165178, 3)
010 (351649, 3)
013 (369077, 3)
012 (382414, 3)
006 (408709, 3)
023 (137646, 3)
022 (337602, 3)
008 (418989, 3)
020 (371496, 3)
021 (302247, 3)
009 (154464, 3)
025 (231729, 3)
019 (297945, 3)
018 (322271, 3)
024 (170534, 3)
026 (195172, 3)
027 (158584, 3)


In [22]:
print(combined_data['006'][0][1])

['standing' 'standing' 'standing' ... 'sitting' 'sitting' 'sitting']


In [24]:
import raw_data_processing
raw_data_processing.store_pickle(combined_data, 'pickled_datasets/harth')

In [26]:
def process_HARTH_thigh_acc_data(data_folder_path):
    combined_data = []
    for filename in os.listdir(data_folder_path):
            file_path = os.path.join(data_folder_path, filename)
            df = pd.read_csv(file_path, sep=',', header=0, na_filter="NaN")
            df['User-ID'] = filename[1:4]
            combined_data.append(df)
    df = pd.concat(combined_data)
    df = df.replace(1, "walking")
    df = df.replace(2, "running")
    df = df.replace(3, "shuffling")
    df = df.replace(4, "stairs (ascending)")
    df = df.replace(5, "stairs (descending)")
    df = df.replace(6, "standing")
    df = df.replace(7, "sitting")
    df = df.replace(8, "lying")
    df = df.replace(13, "cycling (sit)")
    df = df.replace(14, "cycling (stand)")
    df = df.replace(130, "cycling (sit, inactive)")
    df = df.replace(140, "cycling (stand, inactive)")
    back_data = df.loc[:, ['User-ID', 'thigh_x', 'thigh_y', 'thigh_z', 'label']]
    harth_user = back_data["User-ID"].unique()


    user_datasets = {}
    for user in harth_user:
        user_extract = back_data[back_data['User-ID'] == user]
        data = user_extract[['thigh_x', 'thigh_y', 'thigh_z']].values
        labels = user_extract["label"].values
        print(f"{user} {data.shape}")
        user_datasets[user] = [(data, labels)]
    return user_datasets

combined_back_data = process_HARTH_thigh_acc_data("test_run/original_datasets/harth")

016 (355418, 3)
017 (366609, 3)
029 (178716, 3)
015 (418392, 3)
014 (366487, 3)
028 (165178, 3)
010 (351649, 3)
013 (369077, 3)
012 (382414, 3)
006 (408709, 3)
023 (137646, 3)
022 (337602, 3)
008 (418989, 3)
020 (371496, 3)
021 (302247, 3)
009 (154464, 3)
025 (231729, 3)
019 (297945, 3)
018 (322271, 3)
024 (170534, 3)
026 (195172, 3)
027 (158584, 3)


In [6]:
raw_data_processing.store_pickle(hhar_df, 'pickled_datasets/hhar_HAR')